In [1]:
# tao df2
import re
import pandas as pd

path = 'C:/Users/ngocb/Downloads/test_rex.xlsx'
df_trans = pd.read_excel(path)
arr_set_data = set()
num_row_data=0
num_row_process = 0
for col in df_trans.columns:
	df_trans[col] = df_trans[col].astype(str)
	lst_string_column = df_trans[col].to_list()
	num_row_data += len(lst_string_column)
	string = '\n'.join(lst_string_column)
	#thiết lập pattern
	#pattern = re.compile(r'^(?:(?:\w|\s|\b|\n){0,2})(?:(?:[0-9]{2}\/){2}[0-9]{2}[\s][0-9:]+[\s][A-Z]+[^\S][0-9]+[-0-9]{2}[\s]+(?:F|P)[\s][0-9]+[\s][0-9A-Z]+[\s]*(?:VN07)[\s]+(?:[0-9A-Z\s\/#\\]){2,8}[\s]+[0-9-.]*[\s]*(?:EA|CS|NA|N\/A|#NA|#N\/A){0,1}(?:\s|\n|\b){0,50}(?:Tech=){1}[A-Z0-9\s]{10})', re.MULTILINE)
	pattern = re.compile(r'^(?=.*?(?:\d{2}\/){2}(?:\d{2}))(?=.*?(?:(?:\d{2}:){2}\d{2})(?=[\sA-Z]))(?=.*?(?:\d+-\d{1})(?=\s+(?:F|P)))(?=.*?(?:F|P)(?=\s\d{6,12}))(?=.*?(?:\d{6,12})(?=\s{1,5}[A-Z0-9]{6,14}))(?=.*?(?:[A-Z0-9]{6,14})(?=\s{1,20}VN07))(?=.*?(?<=VN07)(?:\s+[A-Z0-9#\/\\\s]{2,14})).{30,}(?:\s|\n|\b){0,50}(?:Tech=){1}[A-Z0-9]{2,15}', re.MULTILINE)
	pattern_check_right = re.compile(r'(?=.*?(?<=VN07)(;[A-Z0-9#\/\\\s]+))(?=.*?([0-9.,]+)(?=;(?:CS|EA)))(?=.*?(?<=(?:CS|EA))(;[A-Z0-9]{2,15}))')
	patern_check_left = re.compile(r'(?=.*?(?<=[A-Z])(;\d+-\d{1}))(?=.*?(?<=(?:\d{2}\/){2}\d{2})(;(?:\d{2}:){2}\d{2}))(?=.*?(?<=(?:F|P))(;\d{2,20}))(?=.*?(?:\d+;)(?=[A-Z0-9]*))')

	list_trans = pattern.findall(string)
	list_trans_remove_duplicate = set(list_trans)
	num_row_process += len(list_trans)
	for i, line in enumerate(list_trans_remove_duplicate):
		#bỏ ký tự \n, Tech=
		row = re.sub(r'(\n|Tech=)', '', line).strip()
		#tìm index VN07
		index_vn07 = re.search(r'VN07', row).span()[0]
		#cắt chuỗi thành 2 phần trái, phải. để xử lý space location
		left_row = row[:index_vn07]
		right_row = row[index_vn07:]
		# with left
		left = re.sub(r'\s{1,}', ';', left_row)
		#with right
		# right1 = re.sub(r'(?<=[A-Z0-9])(\s{1})(?=[0-9A-Z])', '', right_row)
		right2 = re.sub(r'\s{2,}', ';', right_row)
		right = re.sub(r'\s{1,}', '', right2)
		#thêm đơn vị cho RPM
		# regex tìm chữ P theo sau nó phải là dấu ; và trước nó cũng là dấu ;
		# regex tìm chữ F theo sau nó phải là dấu ; và trước nó cũng là dấu ;. Trường hợp CASXFR không có đơn vị
		pattern_p = re.compile(r'(?:(?<=;)(P)(?=;))')
		pattern_f = re.compile(r'(?:(?<=;)(F)(?=;))')
		match_p = pattern_p.search(left)
		match_f = pattern_f.search(left)
		arr_right = right.split(';')
		if (len(arr_right) == 4) and (match_p is not None):
			arr_right.insert(-1, 'EA')
			right = ';'.join(arr_right)
		elif (len(arr_right) == 4) and (match_f is not None):
			arr_right.insert(-1, 'CS')
			right = ';'.join(arr_right)
		#ket qua
		string_final = left + right
		#check kết quả thêm data vào arr kết quả
		match_right = pattern_check_right.match(right)
		match_left = patern_check_left.match(left)
		if match_left and match_right:
			arr_set_data.add(string_final)
	   
print('Tổng số dòng data ban đầu: ', num_row_data)
print('Tổng số dòng cần xử lý: ', num_row_process)
print('Tổng số dòng đã xử lý: ', len(arr_set_data))
print('Tổng số dòng data trùng nhau: ', num_row_process - len(arr_set_data))
arr_set_data

Tổng số dòng data ban đầu:  5240075
Tổng số dòng cần xử lý:  807865
Tổng số dòng đã xử lý:  801949
Tổng số dòng data trùng nhau:  5916


{'11/04/24;07:46:29;PICKUL;0010009772008953020-7;F;80703695;43060395CE;VN07;FC15A;50;CS;BID014',
 '11/17/24;22:12:24;DEPSUL;0010009772008999862-5;F;80796702;43220395Z2;VN07;CONT055;48;CS;BID277',
 '11/29/24;15:25:19;PICKUL;0010009772008998093-4;F;80796697;43340395Z2;VN07;FF32A;48;CS;BID031',
 '11/02/24;07:45:09;DEPSUL;0010009772008954967-4;F;80773336;43070395C9;VN07;FK07E;40;CS;BID348',
 '11/02/24;04:55:23;LINTAK;0010009772008954963-6;F;80773336;43070395C9;VN07;DF;40;CS;BID401',
 '11/18/24;05:34:21;LINTAK;0010009772008974809-1;F;80809266;43220395N0;VN07;AK;40;CS;BID125',
 '11/19/24;02:41:46;PICKUL;0010009772009012568-3;F;80761562;43230395A5;VN07;STDK;48;CS;BID328',
 '11/12/24;13:07:03;PICKUL;0010009772009009870-3;F;80779903;43170395A0;VN07;CONT036;48;CS;BID513',
 '11/02/24;06:40:41;LINTAK;0010009772008978200-2;F;80809581;43060395L2;VN07;AE;68;CS;CC8795',
 '11/06/24;19:17:46;MANLID;0030009772002767767-8;P;21074548;0395411788;VN07;N/A;14000;EA;BID413',
 '11/14/24;19:09:48;PICKUL;00100097

In [2]:
import pandas as pd
lst_total_2 = []
for line in arr_set_data:
    lst_line = line.split(';')
    lst_total_2.append(lst_line)

# transposed = list(zip(*lst_total))

columns = ['date', 'time', 'type', 'ulid', 'class', 'gcas', 'batch', 'plan', 'location', 'qty', 'unit', 'T#']
df_178687 = pd.DataFrame(lst_total_2, columns=columns)
# df_178687.to_excel('df.xlsx')



In [3]:
df_178687.sample(10)

,date,time,type,ulid,class,gcas,batch,plan,location,qty,unit,T#
66638,11/22/24,16:59:55,PICKUL,0010009772008686805-1,F,80783913,43270395R2,VN07,CONT061,45,CS,BID511
605627,11/17/24,00:03:56,DEPSUL,0010009772008989774-4,F,80819031,43190395F1,VN07,C046,44,CS,BID511
59221,11/30/24,00:58:56,PICKUL,0010009772008990756-6,F,80809264,43340395N2,VN07,CONT055,40,CS,BID379
707290,11/01/24,08:28:54,PICKUL,0030009772002763725-2,P,21100796,0395410929,VN07,FG15F,270,EA,BID844
255694,11/16/24,04:40:13,LINTAK,0010009772009020641-2,F,80755744,43200395E4,VN07,DQ,44,CS,BID101
24014,11/14/24,21:18:42,PICKUL,0010009772009007874-3,F,80774738,43190395DF,VN07,WH2FD,80,CS,BID297
740518,11/29/24,02:37:51,PICKUL,0010009772008990597-5,F,80809266,43330395N3,VN07,FE27A,40,CS,BID347
490414,11/04/24,23:30:45,PICKUL,0010009772008972266-4,F,80815423,43040395P0,VN07,FH01D,32,CS,BID909
612512,11/17/24,01:44:23,DEPSUL,0010009772009028111-2,F,80815422,43200395P1,VN07,DA19B1,27,CS,BID909
352063,11/12/24,04:10:29,LINTAK,0010009772008996851-2,F,80779858,43170395O0,VN07,AT,45,CS,BID054


In [4]:
df_sort = df_178687.sort_values(['date', 'time'], ascending=True)
df_sort.tail(10)

,date,time,type,ulid,class,gcas,batch,plan,location,qty,unit,T#
367833,12/01/24,05:57:45,LINTAK,0010009772008990914-0,F,80809262,43350395N2,VN07,AK,40,CS,BID064
358971,12/01/24,05:57:48,LINTAK,0010009772008990913-3,F,80809262,43350395N2,VN07,AK,40,CS,BID064
255048,12/01/24,05:57:50,DEPSUL,0010009772008990914-0,F,80809262,43350395N2,VN07,STAK,40,CS,BID064
416304,12/01/24,05:57:50,DEPSUL,0010009772008990913-3,F,80809262,43350395N2,VN07,STAK,40,CS,BID064
146436,12/01/24,05:58:05,PICKUL,0010009772008990913-3,F,80809262,43350395N2,VN07,STAK,40,CS,BID384
528715,12/01/24,05:58:05,PICKUL,0010009772008990914-0,F,80809262,43350395N2,VN07,STAK,40,CS,BID384
381279,12/01/24,05:58:16,LINTAK,0010009772008991987-3,F,80813651,43360395L0,VN07,AE,50,CS,BID182
305796,12/01/24,05:58:26,DEPSUL,0010009772008991987-3,F,80813651,43360395L0,VN07,STAE,50,CS,BID182
132742,12/01/24,05:59:52,DEPSUL,0010009772008990914-0,F,80809262,43350395N2,VN07,FI25A,40,CS,BID384
156233,12/01/24,05:59:52,DEPSUL,0010009772008990913-3,F,80809262,43350395N2,VN07,FI25A,40,CS,BID384


In [9]:
check_columns = df_178687['class'].unique()
check_columns.sort()
check_columns

array(['F', 'P'], dtype=object)

In [10]:
df_178687[df_178687['ulid'] == '0010009772008759523-9']
#0010009772008683057-7

,date,time,type,ulid,class,gcas,batch,plan,location,qty,unit,T#


### So sánh 2 dataframe

In [53]:
# df_dff = pd.merge(left=df_180430,
#                   right=df_178687,
#                   on=['Time', 'Ulid'],
#                   how='left',
#                   indicator=True)


In [54]:
# df_khac = df_dff[df_dff['_merge'].isin(['left_only'])]
# # df_khac.to_excel('df_dff.xlsx')
# df_khac